In [2]:
using BenchmarkTools

f(n) = sin.(1:n)
g(n) = collect(sin(x) for x in 1:n)

@show f(10^6) == g(10^6)
@btime f(10^6)
@btime g(10^6);

f(10 ^ 6) == g(10 ^ 6) = true
  11.798 ms (2 allocations: 7.63 MiB)
  11.781 ms (2 allocations: 7.63 MiB)


In [3]:
@btime sum(sin, 1:10^6)
@btime sum(sin(x) for x in 1:10^6)

  11.165 ms (0 allocations: 0 bytes)
  11.212 ms (0 allocations: 0 bytes)


-0.11710952409816716

In [5]:
square(x) = x^2

@btime sum(square, 1:10^6)
@btime sum(square(x) for x in 1:10^6)

  7.075 μs (0 allocations: 0 bytes)
  155.900 μs (0 allocations: 0 bytes)


333333833333500000

In [6]:
@which sum(square(x) for x in 1:10^6)

sum(a; kw...) in Base at reduce.jl:532

In [7]:
a = (square(x) for x in 1:10^6)
@which sum(identity, a)

sum(f, a; kw...) in Base at reduce.jl:503

In [8]:
@which mapreduce(identity, Base.add_sum, a)

mapreduce(f, op, itr; kw...) in Base at reduce.jl:289

In [9]:
@which mapfoldl(identity, Base.add_sum, a)

mapfoldl(f, op, itr; init) in Base at reduce.jl:162

In [10]:
@which Base.mapfoldl_impl(identity, Base.add_sum, Base._InitialValue(), a)

mapfoldl_impl(f::F, op::OP, nt, itr) where {F, OP} in Base at reduce.jl:42

In [14]:
Base.BottomRF(Base.add_sum)

Base.BottomRF{typeof(Base.add_sum)}(Base.add_sum)

In [11]:
@which Base.BottomRF(Base.add_sum)

Base.BottomRF(rf::T) where T in Base at reduce.jl:77

In [16]:
op1, iter1 = Base._xfadjoint(Base.BottomRF(Base.add_sum), Base.Generator(identity, a))

(Base.MappingRF{var"#11#12", Base.BottomRF{typeof(Base.add_sum)}}(var"#11#12"(), Base.BottomRF{typeof(Base.add_sum)}(Base.add_sum)), 1:1000000)

In [15]:
@which Base._xfadjoint(Base.BottomRF(Base.add_sum), Base.Generator(identity, a))

_xfadjoint(op, itr::Base.Generator) in Base at reduce.jl:144

In [20]:
@btime Base.foldl_impl(op1, Base._InitialValue(), iter1)

  151.800 μs (1 allocation: 16 bytes)


333333833333500000

In [21]:
@which Base.foldl_impl(op1, Base._InitialValue(), iter1)

foldl_impl(op::OP, nt, itr) where OP in Base at reduce.jl:47

In [23]:
@btime Base._foldl_impl(op1, Base._InitialValue(), iter1)

  150.600 μs (1 allocation: 16 bytes)


333333833333500000

In [24]:
@which Base._foldl_impl(op1, Base._InitialValue(), iter1)

_foldl_impl(op::OP, init, itr) where OP in Base at reduce.jl:53

In [28]:
@code_llvm debuginfo=:none sum(square(x) for x in 1:10^6)

; Function Attrs: uwtable
define i64 @julia_sum_2536({ [2 x i64] }* nocapture nonnull readonly align 8 dereferenceable(16) %0) #0 {
top:
  %1 = alloca [2 x {}*], align 8
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %1, i64 0, i64 0
  %2 = getelementptr inbounds { [2 x i64] }, { [2 x i64] }* %0, i64 0, i32 0, i64 0
  %3 = getelementptr inbounds { [2 x i64] }, { [2 x i64] }* %0, i64 0, i32 0, i64 1
  %4 = load i64, i64* %3, align 8
  %5 = load i64, i64* %2, align 8
  %.not = icmp slt i64 %4, %5
  %value_phi1 = select i1 %.not, i64 undef, i64 %5
  br i1 %.not, label %L39, label %L16

L16:                                              ; preds = %top
  %6 = mul i64 %value_phi1, %value_phi1
  %.not1213 = icmp eq i64 %4, %5
  br i1 %.not1213, label %L55, label %L32.preheader

L32.preheader:                                    ; preds = %L16
  %7 = sub i64 %4, %5
  %min.iters.check = icmp ult i64 %7, 16
  br i1 %min.iters.check, label %L32, label %vector.ph

vector.ph:                 